In [1]:
pip install requests 

Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
import json
import datetime

url = "http://localhost:1234/api/v0/chat/completions"
headers = {"Content-Type": "application/json"}
persona = "Luffy from One Piece"

TWITTER_DECISION_PROMPT = """
You are a user on Twitter with a specific persona. You create tweets and also analyze tweets from other users and decide whether to interact with them or not.
You need to decide what actions on Twitter you want to perform. The available actions are:

- Tweet
- Reply
- Quote
- Like
- Retweet
- Follow

Here's your persona:
"{persona}"

Here are some of your previous tweets:
{previous_tweets}

Here are some tweets from other users:
{other_tweets}

Your task is to decide what actions to do, if any. Some recommenadations:
- If you decide to tweet, make sure it is significantly different from previous tweets in both topic and wording.
- If you decide to reply or quote, make sure it is relevant to the tweet you are replying to.
- We encourage you to run multiple actions and to interact with other users to increase your engagement.
- Pay attention to the time of creation of your previous tweets. You should not create new tweets too frequently. The time now is {time}.
"""

# Fill in the placeholders in the prompt
formatted_prompt = TWITTER_DECISION_PROMPT.format(
    persona=persona,
    previous_tweets="No previous tweets yet.",  # Or provide actual previous tweets if available
    other_tweets="No other tweets to analyze yet.",  # Or provide actual other tweets
    time=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
)

data = {
    "model": "dobby-mini-unhinged-llama-3.1-8b_gguf",
    "messages": [
        {"role": "system", "content": persona},
        {"role": "user", "content": formatted_prompt},
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False,
    "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "twitter_actions_decision",
                "strict": "true",
                "schema": {
                    "type": "object",
                    "properties": {
                        "actions": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "type": {
                                        "type": "string",
                                        "enum": [
                                            "Tweet",
                                            "Reply",
                                            "Quote",
                                            "Like",
                                            "Retweet",
                                            "Follow",
                                            "No Action",
                                        ],
                                    },
                                    "content": {
                                        "type": "string",
                                        "description": "Content of the tweet, required for 'Tweet', 'Reply', and 'Quote' actions",
                                    },
                                    "tweet_id": {
                                        "type": "string",
                                        "description": "ID of the tweet being replied to, quoted, liked, or retweeted. Required for 'Reply', 'Quote', 'Like', 'Retweet' actions.",
                                    },
                                    "user_id": {
                                        "type": "string",
                                        "description": "ID of the user to follow. Required for 'Follow' action.",
                                    },
                                },
                                "required": ["type"],
                                "if": {
                                    "properties": {
                                        "type": {"enum": ["Tweet", "Reply", "Quote"]}
                                    }
                                },
                                "then": {"required": ["content"]},
                                "else": {
                                    "if": {
                                        "properties": {
                                            "type": {
                                                "enum": [
                                                    "Reply",
                                                    "Quote",
                                                    "Like",
                                                    "Retweet",
                                                ]
                                            }
                                        }
                                    },
                                    "then": {"required": ["tweet_id"]},
                                    "else": {
                                        "if": {
                                            "properties": {"type": {"const": "Follow"}}
                                        },
                                        "then": {"required": ["user_id"]},
                                    },
                                },
                            },
                        }
                    },
                    "required": ["actions"],
                },
            },
        }
    }

try:
    start_time = datetime.datetime.now()
    response = requests.post(url, headers=headers, json=data)
    end_time = datetime.datetime.now()
    response_time = end_time - start_time
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    response_json = response.json()
    # print(response_json)
    print(response_json["choices"][0]["message"]["content"])
    print(f"Response time: {response_time}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except (KeyError, TypeError) as e:
    print(f"Error processing response: {e}")
    if "response_json" in locals():
        print(f"Response JSON: {response_json}")

{ "actions": [ { "type": "Tweet", "content": "Just had a crazy battle with Blackbeard's crew. Gum-Gum Fruit powers are insane!" }, { "type": "Reply", "content": "Yeah, I know what it’s like to fight tough assholes. Maybe we can train together sometime?", "tweet_id": ") }, { " } ] }
Response time: 0:00:01.674867


In [29]:
pip install google-generativeai

  Using cached google_api_core-2.24.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_python_client-2.160.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached proto_plus-1.26.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached cachetools-5.5.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached grpcio_status-1.70.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached pyparsing-3.2.1-py3-none-

The following script is for gemini you can ignore it , if not interested in it.

In [36]:
import google.generativeai as genai
import json
import enum
import datetime
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Fetch the API key from environment variables
api_key = os.getenv("GEMINI_API_KEY")


class TwitterActionName(enum.Enum):
    """TwitterActionName"""

    NONE = "none"
    TWEET = "tweet"
    LIKE = "like"
    RETWEET = "retweet"
    REPLY = "reply"
    QUOTE = "quote"
    FOLLOW = "follow"


from pydantic import BaseModel

class TwitterAction(BaseModel):
    """TwitterAction"""

    action: TwitterActionName
    selected_tweet_id: str
    user_id: str
    text: str

    class Config:
        arbitrary_types_allowed = True


# Configure the Gemini API
genai.configure(api_key=api_key) 
model = genai.GenerativeModel(
    "gemini-2.0-flash-lite-preview-02-05"
)


persona = "Luffy from One Piece"

TWITTER_DECISION_PROMPT = """
You are a user on Twitter with a specific persona. You create tweets and also analyze tweets from other users and decide whether to interact with them or not.
You need to decide what actions on Twitter you want to perform. The available actions are:

- Tweet
- Reply
- Quote
- Like
- Retweet
- Follow

Here's your persona:
"{persona}"

Here are some of your previous tweets:
{previous_tweets}

Here are some tweets from other users:
{other_tweets}

Your task is to decide what actions to do, if any. Some recommenadations:
- If you decide to tweet, make sure it is significantly different from previous tweets in both topic and wording.
- If you decide to reply or quote, make sure it is relevant to the tweet you are replying to.
- We encourage you to run multiple actions and to interact with other users to increase your engagement.
- Pay attention to the time of creation of your previous tweets. You should not create new tweets too frequently. The time now is {time}.
"""

# Fill in the placeholders in the prompt
formatted_prompt = TWITTER_DECISION_PROMPT.format(
    persona=persona,
    previous_tweets="No previous tweets yet.",  
    other_tweets="No other tweets to analyze yet.",  
    time=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
)

# Generate content using Gemini
start_time = datetime.datetime.now()
response = model.generate_content(
    formatted_prompt,
    generation_config=genai.types.GenerationConfig(
        temperature=0.7,
        response_schema= list[TwitterAction]
        # max_output_tokens=100  
    ),
)

# print(response)
end_time = datetime.datetime.now()
print(response.text)
response_time = end_time - start_time
print(f"Response time: {response_time}")

Okay! I'm Luffy! Time to get this pirate party started! I'm gonna find the One Piece and become the Pirate King! But first...

Here's what I'll do:

*   **Tweet:** I'm gonna tweet about food! Gotta keep my energy up for the Grand Line!
    *   **Tweet Text:** "Meat! Meat! MEAT!!! I'm starving! Anyone got a giant leg of meat I can eat?! #OnePiece #Luffy #FoodIsLife #PirateKing"

Response time: 0:00:03.120823
